<a href="https://colab.research.google.com/github/patch0000/TwitterAPI_sample/blob/main/check_FF_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Twitterで自分がフォローしているフォロー先とフォロワーの一覧を取得してCSV及び、Googleスプレッドシートに保存するPythonスクリプト

**前提条件**  
TwitterのDeveloper Account登録をしてBearer Tokenを取得済である事

**大まかな流れ**

(1)自分のusername(@hogeのhogeの部分)からuser_id(ユーザを一意に特定する番号)を得る

(2)user_idとBearer Tokenを使って自分とFF関係のユーザの一覧を得てCSVに保存する

(3)CSVから取得したフォロー先ユーザの情報をGoogle Spread Sheetに保存する


## (1)自分のusernameからuser_idを取得

参考情報

Twitter社が提供している公式サンプル

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/User-Lookup/get_users_with_bearer_token.py

In [ ]:
import requests
import os
import json

# ★TODO：自分のBearer Tokenに差し替えてください。
bearer_token = "xxxxxxxxx"

def create_url():
    # ★TODO：XXXXXXを自分のusernames(@hogehogeのhogehogeの部分)に差し替えてください。
    usernames = "usernames=XXXXXX"
    user_fields = "user.fields=description,created_at"
    url = "https://api.twitter.com/2/users/by?{}&{}".format(usernames, user_fields)
    return url

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2UserLookupPython"
    return r

def connect_to_endpoint(url):
    response = requests.request("GET", url, auth=bearer_oauth,)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def main():
    url = create_url()
    json_response = connect_to_endpoint(url)
    print(json.dumps(json_response, indent=4, sort_keys=True))

if __name__ == "__main__":
    main()

'''
実行すると、以下のように表示されるので"id": "xxxxxxxxxxxxxxxxxxxx"のxxxxxxxxxxxxの部分をメモっておく
200
{
    "data": [
        {
            "created_at": "yyyy-mm-ddThh:mm:ss.000Z",
            "description": "xxxxxxxxxxxxxxxxxxx",
            "id": "xxxxxxxxxxxxxxxxxxxxxxxxx",
            "name": "XXXXX",
            "username": "XXXX"
        }
    ]
}
'''

## (2)user_idとBearer Tokenを使ってフォロー先のユーザ情報を得る

参考情報
Twitter社が提供している公式サンプル

https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/main/Follows-Lookup/followers_lookup.py

https://developer.twitter.com/en/docs/twitter-api/users/follows/api-reference/get-users-id-following


In [ ]:
import requests
import os
import json
import pandas as pd

# ★TODO：自分のBearer Tokenに差し替えてください。
bearer_token = "xxxxxxxx"

def create_url():
    # ★TODO：xxxxxxxxxxxxxの部分を(1)で取得した自分のuser_idに差し替えてください。
    # また、このAPIは連続して実行すると
    # 「Request returned an error: 429 Rate limit exceeded」とエラーになる事があります。
    user_id = xxxxxxxxxxxx
    return "https://api.twitter.com/2/users/{}/following".format(user_id)

def get_params(pagination_token=""):
    if pagination_token != "":
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000", "pagination_token": pagination_token}
    else:
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000"}

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def save_range_list(json_response, range_list):
  cell_list = []
  for v in json_response['data']:
    # userを一意に特定できる不変のID
    cell_list.append(v['id'])
    # プロフィール画面などに出て来る名称
    cell_list.append(v['name'])
    # Twitterアカウントの@hogehogeのhogehogeの部分
    cell_list.append(v['username'])
    cell_list.append(v['public_metrics']['followers_count'])
    cell_list.append(v['public_metrics']['listed_count'])
    cell_list.append(v['public_metrics']['tweet_count'])
    range_list.append(cell_list)
    cell_list = []

def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    #print(json.dumps(json_response, indent=4, sort_keys=True))

    range_list = [['id', 'name', 'username', 'followers_count', 'listed_count','tweet_count']]
    save_range_list(json_response, range_list)

    if json_response.get("meta") and json_response.get("meta").get("next_token"):
      print(json.dumps(json_response.get("meta"), indent=4, sort_keys=True))
    
    while (json_response.get("meta") and json_response.get("meta").get("next_token")):
      params = get_params(json_response['meta']['next_token'])
      json_response = connect_to_endpoint(url, params)
      save_range_list(json_response, range_list)
      if json_response.get("meta") and json_response.get("meta").get("next_token"):
        print(json.dumps(json_response.get("meta"), indent=4, sort_keys=True))

    # /content/df_following.csvとして保存する
    df = pd.DataFrame(range_list)
    df.to_csv("/content/df_following.csv", header=False, index=False)


if __name__ == "__main__":
    main()

フォロワー

In [ ]:
import requests
import os
import json
import pandas as pd

# ★TODO：自分のBearer Tokenに差し替えてください。
bearer_token = "xxxxxxxx"

def create_url():
    # ★TODO：xxxxxxxxxxxxxの部分を(1)で取得した自分のuser_idに差し替えてください。
    # また、このAPIは連続して実行すると
    # 「Request returned an error: 429 Rate limit exceeded」とエラーになる事があります。
    user_id = xxxxxxxxxxxxx
    return "https://api.twitter.com/2/users/{}/followers".format(user_id)

def get_params(pagination_token=""):
    if pagination_token != "":
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000", "pagination_token": pagination_token}
    else:
      return {"user.fields": "id,name,username,public_metrics", "max_results": "1000"}

def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FollowersLookupPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.request("GET", url, auth=bearer_oauth, params=params)
    # print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Request returned an error: {} {}".format(
                response.status_code, response.text
            )
        )
    return response.json()

def save_range_list(json_response, range_list):
  cell_list = []
  for v in json_response['data']:
    # userを一意に特定できる不変のID
    cell_list.append(v['id'])
    # プロフィール画面などに出て来る名称
    cell_list.append(v['name'])
    # Twitterアカウントの@hogehogeのhogehogeの部分
    cell_list.append(v['username'])
    cell_list.append(v['public_metrics']['followers_count'])
    cell_list.append(v['public_metrics']['listed_count'])
    cell_list.append(v['public_metrics']['tweet_count'])
    range_list.append(cell_list)
    cell_list = []

def main():
    url = create_url()
    params = get_params()
    json_response = connect_to_endpoint(url, params)
    #print(json.dumps(json_response, indent=4, sort_keys=True))

    range_list = [['id', 'name', 'username', 'followers_count', 'listed_count','tweet_count']]
    save_range_list(json_response, range_list)
    
    while (json_response.get("meta") and json_response.get("meta").get("next_token")):
      params = get_params(json_response['meta']['next_token'])
      json_response = connect_to_endpoint(url, params)
      save_range_list(json_response, range_list)

    # /content/df_follower.csvとして保存する
    df = pd.DataFrame(range_list)
    df.to_csv("/content/df_follower.csv", header=False, index=False)


if __name__ == "__main__":
    main()

## (3)Google スプレッドシートに保存する

参考情報：
Google スプレッドシートを操作するためのライブラリgspread  
https://docs.gspread.org/en/latest/index.html

In [ ]:
# ディフォルトで入っているgspreadはVersionが古い。
# version 3.7以上でないと「AttributeError: 'Worksheet' object has no attribute 'update'」
# とエラーがでてworksheet.updateに失敗する。そのためアップデートを行う。
# ★TODO：gspreadが既に取り込まれていた場合はこのセル実行後に上段メニューから「ランタイム」→「ランタイムを再起動」で再起動する事
! pip install -U gspread

In [ ]:
import gspread
import pandas as pd

In [ ]:
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread

auth.authenticate_user()

from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)


In [ ]:
workbook = ""
workbook = gc.create('Twitter_FF_check')
worksheet = workbook.get_worksheet(0)

# CSVの読み込み
df = pd.read_csv("/content/df_following.csv")
df2 = pd.read_csv("/content/df_follower.csv")

#df.info()
#df2.info()

following_res = pd.merge(df, df2, on='id', how='left')
following_res['url_following'] = "https://twitter.com/" + following_res['username_x']

follower_res = pd.merge(df2, df, on='id', how='left')
follower_res['url_follower'] = "https://twitter.com/" + follower_res['username_x']

# print(res.head)
# print(res.columns)
#res.info()
#res = res.astype({'_merge': str})

#idを数値のままにしておくと巨大すぎてGoogleスプレッドシートで末尾が000に切り捨てられてしまうため文字列に変換
follower_res = follower_res.astype({'id': str})
following_res = following_res.astype({'id': str})

# 空の項目(NaN)があると「APIError: {'code': 400, 'message': 'Invalid JSON payload received. Unexpected token～}」
# のエラーがでて失敗するのでNaNを埋める
following_res.fillna('', inplace=True)
worksheet.update([following_res.columns.values.tolist()] + following_res.values.tolist())

follower_res.fillna('', inplace=True)
worksheet2 = workbook.add_worksheet(title="follower", rows=len(follower_res), cols=len(follower_res.columns)  )
worksheet2.update([follower_res.columns.values.tolist()] + follower_res.values.tolist())



スプレッドシートは( https://drive.google.com/drive/my-drive) に出来ます。  

Colabのランタイムをリセットしないとゴミ箱に入れたファイルに書き込んだり、
どのスプレッドシートに書き込まれたのかわからなくなってしまったり、
Requested entity was not foundとエラーが出る事があります。
その場合はランタイムを再起動したり、以下の方法で捜し出してください。


上のセルを実行した際に  
{'spreadsheetId': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx',  
・・・と表示されると思いますが、  
スプレッドシートのURLのyyyyyyyyyyyyyyyyyyyyyyyyyyyの部分を置き換えると直接アクセスできます。  
https://docs.google.com/spreadsheets/d/yyyyyyyyyyyyyyyyyyyyyyyyyyy/edit#gid=0

